In [175]:
import pandas as pd
import numpy as np

L_classes_sauv = [["2nde",17,4.5],["2nde_ND",1,3],["SL",2,1.5],["1ES",12,1.5],["1SPC",8,4],["TES",12,1.5],["TSPC",4,6],["TSI",2,2],["1STI",3,6],["TSTI",3,6]]
L_classes = [["2nde",17,4.5],["2nde_ND",1,3],["SL",3,1.5],["1ES",12,1.5],["1SPC",8,4],["TES",12,1.5],["TSPC",4,6],["TSI",2,2],["1STI",3,6],["TSTI",3,6]]
L_qu = ["Combien de 2nde ? ","2nde non dédoublée ? ","Combien de SL ? ","Combien de 1ES ? ","Combien de 1SPC ? ","Combien de TES ? ","Combien de TSPC ? ","Combien de T-SI ? ","Combien de 1STI ? ","Combien de TSTI ? "]

nb, noms, classes = [],[],[]

for i in range(len(L_qu)):
    nb.append(0)

def init() :
    tableau = pd.DataFrame({'Noms' : [],'2nde' : [],'2nde_ND' : [],'SL' : [],'1ES' : [],'1SPC' : [],'TES' : [],'TSPC' : [],'TSI' : [],\
                        '1STI' : [],'TSTI' : [],'Service_dû' : [],'Service_effectif' : [],'Diff' : []},\
                       columns=['Noms','2nde','2nde_ND','SL','1ES','1SPC','TES','TSPC','TSI','1STI','TSTI','Service_dû','Service_effectif','Diff'])
    return tableau

tableau = init()
    
def voeux(tableau) :
    row = []
    nom = input("Nom ? ")
    service_du = int(input("Service ? "))
    service_eff = 0
    row += [nom]
    for i in range(len(L_classes)) :
        nb[i] = int(input(L_qu[i]))
        L_classes[i][1] -= nb[i]
        row += [nb[i]]
        service_eff += nb[i]*L_classes[i][2]
    row += [service_du]
    row += [service_eff]
    row += [service_eff-service_du]
    tableau.loc[len(tableau)] = row
    return tableau

def raz(tableau) :
    for i in range(1,len(tableau)) :
        tableau[i]=[]
        
def lire() :
    df = pd.read_csv('recap.csv',sep=',')
    df.apply(pd.to_numeric, errors='ignore')
    return df

def problemes(tableau) :
    print()
    s , NB , pofs_oui , profs_non  = [] , [] , [] , []
    b = c = pd.DataFrame({'A' : []})
    b = pd.DataFrame(L_classes_sauv)
    b = b[1]
    c = tableau.sum(axis = 0, skipna = True)[1:-2]
    c = pd.to_numeric(c)
    for i in range(len(L_classes_sauv)) :
        if c[i]<b[i] :
            print("- Il manque {} {}".format(int(b[i]-c[i]),L_classes_sauv[i][0]))
            sans = tableau[L_classes_sauv[i][0]] == 0
            profs_sans = tableau[sans]['Noms']
            print("Profs n'en ayant pas :",*profs_sans,sep='  ',end="")
            print()
        if c[i]>b[i] :
            print("- Il y a {} {} en trop".format(int(c[i]-b[i]),L_classes_sauv[i][0]))
            avec = tableau[L_classes[i][0]] >= 1
            profs_avec = tableau[avec]['Noms']
            NB = tableau[avec][L_classes_sauv[i][0]]
            print("Profs en ayant : ",end="")
            for k in range(len(profs_avec)) :
                print("{} ({})".format(list(profs_avec)[k],int(list(NB)[k])),end=" ")
            print()
    print()
    print("Profs n'ayant pas leur quota :")
    endessous = tableau["Diff"] < 0
    profs_endessous = tableau[endessous]['Noms']
    nbh1 = tableau[endessous]['Diff']
    for k in range(len(profs_endessous)) :
        print("{} ({} h)".format(list(profs_endessous)[k],list(nbh1)[k]),end=" ")
    print("")
    print("\nProfs avec bcp d'heures :")
    audessus = tableau["Diff"] > 2
    profs_audessus = tableau[audessus]['Noms']
    nbh2 = tableau[audessus]['Diff']
    for k in range(len(profs_audessus)) :
        print("{} ({} h)".format(list(profs_audessus)[k],list(nbh2)[k]),end=" ")

def aj_suppr(tableau) :
    while 1 :
        nom = input("Nom ? ")
        if nom in tableau.values : break
        print("pas dans la liste")
    A,B,C = zip(*L_classes_sauv)
    while 1 :
        classe = input("Classe ? ")
        if classe in A : break
        print("pas dans la liste")
    var = input("Nombre i de classes à ajouter (i>0) ou à retirer (i>0) : ")
    tableau.loc[tableau['Noms'] == nom, classe] += int(var)
    serv_eff = 0
    for e in L_classes :
        serv_eff += tableau.loc[tableau['Noms'] == nom, e[0]]*e[2]
    tableau.loc[tableau['Noms'] == nom, 'Service_effectif'] = serv_eff
    tableau.loc[tableau['Noms'] == nom, 'Diff'] = serv_eff - tableau.loc[tableau['Noms'] == nom, 'Service_dû']
    return tableau

def echange(tableau) :
    while 1 :
        nom = input("Qui donne ? ")
        if nom in tableau.values : break
        print("pas dans la liste")
    A,B,C = zip(*L_classes_sauv)
    while 1 :
        classe = input("Classe ? ")
        if classe in A : break
        print("pas dans la liste")
    if tableau.loc[tableau['Noms']==nom, classe].values[0] == 0 :
        print("Impossible, {} n'a aucune {} !".format(nom,classe))
        return tableau
    while 1 :
        recipiendaire = input("Qui prend ? ")
        if (recipiendaire in tableau.values) or (recipiendaire == 0): break
        print("pas dans la liste")
    while 1 :
        var = input("Combien ? ")
        if var in ['0','1','2','3'] : break
        print("pas une valeur autorisée")
    tableau.loc[tableau['Noms'] == nom, classe] -= int(var)
    if recipiendaire != '0' :
        tableau.loc[tableau['Noms'] == recipiendaire, classe] += int(var)
    serv_eff = 0
    serv_eff_rec = 0
    for e in L_classes :
        serv_eff += tableau.loc[tableau['Noms'] == nom, e[0]]*e[2]
        serv_eff_rec += tableau.loc[tableau['Noms'] == recipiendaire, e[0]]*e[2]
    tableau.loc[tableau['Noms'] == nom, 'Service_effectif'] = serv_eff
    tableau.loc[tableau['Noms'] == nom, 'Diff'] = serv_eff - tableau.loc[tableau['Noms'] == nom, 'Service_dû']
    if recipiendaire != '0' :
        tableau.loc[tableau['Noms'] == recipiendaire, 'Service_effectif'] = serv_eff_rec
        tableau.loc[tableau['Noms'] == recipiendaire, 'Diff'] = serv_eff_rec - tableau.loc[tableau['Noms'] == recipiendaire, 'Service_dû']
    return tableau

def tri(tableau) :
    tableau = tableau.sort_values('Noms')
    tableau.set_index('Noms',inplace=True)
    tableau.reset_index(inplace=True)
    return tableau

def sauve(tableau) :
    tableau.to_csv('recap.csv', index = False)
    #tableau.to_excel("recap.xlsx", index = False)

In [146]:
tableau = voeux(tableau)

Nom ?  Anne
Service ?  15
Combien de 2nde ?  0
2nde non dédoublée ?  0
Combien de SL ?  0
Combien de 1ES ?  2
Combien de 1SPC ?  0
Combien de TES ?  0
Combien de TSPC ?  0
Combien de T-SI ?  0
Combien de 1STI ?  2
Combien de TSTI ?  0


In [176]:
tableau = lire()

In [186]:
tri(tableau)

,Noms,2nde,2nde_ND,SL,1ES,1SPC,TES,TSPC,TSI,1STI,TSTI,Service_dû,Service_effectif,Diff
0,Anne,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,15.0,15.0,0.0
1,Catherine B,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,18.0,18.5,0.5
2,Catherine M,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,18.0,17.0,-1.0
3,David B,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,18.0,7.0,-11.0
4,David C,2.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,18.0,17.5,-0.5
5,Doriane,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,18.0,19.5,1.5
6,Jean-Michel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,0.0
7,Jean-Pierre,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,6.0,0.0
8,Julie,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,18.0,0.0
9,Marc,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,15.0,15.0,0.0


In [180]:
tableau

,Noms,2nde,2nde_ND,SL,1ES,1SPC,TES,TSPC,TSI,1STI,TSTI,Service_dû,Service_effectif,Diff
0,Philippe,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,18.0,19.5,1.5
1,David B,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,18.0,7.0,-11.0
2,Marc,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,3.0,-12.0
3,Julie,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,18.0,0.0
4,Catherine B,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,18.0,18.5,0.5
5,Jean-Michel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,0.0
6,Morgane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,12.0,4.0
7,Catherine M,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,18.0,17.0,-1.0
8,Nicolas,2.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,18.0,20.0,2.0
9,David C,2.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,18.0,17.5,-0.5


In [185]:
problemes(tableau)


- Il manque 2 2nde
Profs n'en ayant pas :  David B  Marc  Jean-Michel  Morgane  Jean-Pierre  Anne
- Il manque 1 1SPC
Profs n'en ayant pas :  Marc  Julie  Jean-Michel  Morgane  Catherine M  Doriane  Marjorie  Jean-Pierre  Anne
- Il manque 3 TES
Profs n'en ayant pas :  Julie  Jean-Michel  Morgane  Catherine M  Jean-Pierre  Anne
- Il y a 1 TSPC en trop
Profs en ayant : Julie (1) Catherine M (1) Doriane (1) Marjorie (1) Jean-Pierre (1) 
- Il y a 1 TSTI en trop
Profs en ayant : Marc (1) Jean-Michel (1) Morgane (2) 

Profs n'ayant pas leur quota :
David B (-11.0 h) Catherine M (-1.0 h) David C (-0.5 h) 

Profs avec bcp d'heures :
Morgane (4.0 h) 

In [183]:
tableau = aj_suppr(tableau)

Nom ?  Marc
Classe ?  TSTI
Nombre i de classes à ajouter (i>0) ou à retirer (i>0) :  1


In [159]:
tableau = echange(tableau)

Qui donne ?  Philippe
Classe ?  2nde_ND
Qui prend ?  Philippe
Combien ?  1


In [98]:
tableau = tableau.drop([5]) #Enlève la ligne numéro i (pas sûr que besoin du tableau=)

In [187]:
sauve(tableau)

In [42]:
tableau.loc[tableau['Noms']=='Marc', '1STI'].values[0]

1.0